1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변 생성 못 할수 있음
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("tax_with_markdown.docx")

document_list = loader.load_and_split(text_splitter)

In [2]:
len(document_list)


225

In [16]:
documment_list[52]

NameError: name 'documment_list' is not defined

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
from langchain_upstage import UpstageEmbeddings
embedding = UpstageEmbeddings(model="embedding-query")

In [5]:
import os

from pinecone import Pinecone

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


/opt/anaconda3/envs/project1122env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [7]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [8]:
retrieved_docs

[]

In [9]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage(model="solar-mini")

In [10]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# [Question]
# {query}
# """

In [11]:
# ai_message = llm.invoke(prompt)

In [12]:
# ai_message.content

In [13]:
from langchain import hub
from langchain.chains import RetrievalQA

prompt = hub.pull("rlm/rag-prompt")

/opt/anaconda3/envs/project1122env/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [15]:
ai_message = qa_chain({"query": query})
ai_message

/var/folders/dr/74f443cx0ljg_9kz1_q1ylv80000gn/T/ipykernel_5065/2125453911.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 87만 6천 5백원이다.'}